In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset

import pickle
import numpy as np

# Load MNIST dataset using pickle
def load_mnist(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

# Specify the path to the MNIST dataset file
mnist_file = 'C:/Users/HP/OneDrive/Desktop/MNIST_data/MNIST_data.pkl'

# Load MNIST data
mnist_data = load_mnist(mnist_file)

# Separate data into training and testing sets
train_images, train_labels = mnist_data[0]
test_images, test_labels = mnist_data[1]

# Convert data to PyTorch tensors
train_images = torch.Tensor(train_images).view(-1, 1, 28, 28)
train_labels = torch.LongTensor(train_labels)
test_images = torch.Tensor(test_images).view(-1, 1, 28, 28)
test_labels = torch.LongTensor(test_labels)

# Create DataLoader for training and testing sets
batch_size = 100
train_dataset = TensorDataset(train_images, train_labels)
test_dataset = TensorDataset(test_images, test_labels)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Define the neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


model = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

"""  FOR FUTURE REFERENCE TO CHECK FOR THE MODEL'S ACCURACY, YET TO DEBUG

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy on the test set: {100 * correct / total}%')
"""

KeyError: 0